<a href="https://colab.research.google.com/github/ykitaguchi77/CongenitalGlaucoma_AI_project/blob/main/Cornea_train_annotated_ellipse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Train model to predict annoted ellipse**

In [1]:
import codecs
import pandas as pd
import os
import numpy as np
import random
import shutil
import re
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
import pandas as pd
from PIL import Image
import tempfile
import time
import glob
import copy
import pickle
import itertools
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
!pip install torch_optimizer
import torch_optimizer as optim
import torch.utils.data as data
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

random_seed = 2 #shuffleのシード

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pd.set_option('display.max_rows', 500)

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 2.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
xml_path = "/content/drive/MyDrive/Deep_learning/Congenital_Glaucoma/glaucoma_cornea_ellipse/annotations.xml"

In [117]:
import xml.etree.ElementTree as ET
import pandas as pd

# XMLデータをパースする
tree = ET.parse(xml_path)
root = tree.getroot()

# イメージの情報を含む行のリストを作成する
image_data = []
for image in root.findall('image'):
    try:
        attributes = image.attrib
        image_id = attributes.get('id', None)
        name = attributes.get('name', None)
        width = attributes.get('width', None)
        height = attributes.get('height', None)
        ellipse = image.find('ellipse')
        label = ellipse.get('label', None)
        occluded = ellipse.get('occluded', None)
        source = ellipse.get('source', None)
        cx = ellipse.get('cx', None)
        cy = ellipse.get('cy', None)
        rx = ellipse.get('rx', None)
        ry = ellipse.get('ry', None)
        rotation = ellipse.get('rotation', None)
        z_order = ellipse.get('z_order', None)
        image_data.append([image_id, name, width, height, label, occluded, source, cx, cy, rx, ry, rotation, z_order])
    except:
        pass

# データフレームに変換する
df = pd.DataFrame(image_data, columns=['image_id', 'name', 'width', 'height', 'label', 'occluded', 'source', 'cx', 'cy', 'rx', 'ry', 'rotation', 'z_order'])

# "nonetype"オブジェクトを削除する
df["rotation"] = df["rotation"].replace({None: 0})

# 患者番号を追加
df["patient"] = [i.split("-")[0] for i in df["name"]]



In [123]:
image_path = "/content/drive/MyDrive/Deep_learning/Congenital_Glaucoma/glaucoma_cornea_ellipse/images/0-1.jpg"
target = df[df["name"]==os.path.basename(image_path)].loc[:, "cx":"rotation"].apply(pd.to_numeric, errors='coerce')
target = target.values.tolist()[0]
target = torch.tensor(target)
target

tensor([312.1800, 312.6500, 109.4500, 108.9500,   0.0000])

In [120]:
class SimpleImageDataset(Dataset):
    def __init__(self, img_list, df, transform):
        self.transform = transform
        self.img_list = img_list
        self.df = df
        self.item_dict = {}

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image_path = self.img_list[idx]
        pilr_image = Image.open(image_path).convert("RGB")
        tensor_image = self.transform(pilr_image)
        target = df[df["name"]==os.path.basename(image_path)].loc[:, "cx":"rotation"].apply(pd.to_numeric, errors='coerce')
        target = target.values.tolist()[0]
        target = torch.tensor(target)
        return tensor_image, target


#少数の画像を可視化
def imshow(inp, title=None):
    """Imshow for Tensor."""

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


#Defining early stopping class

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


#Train models
def train_model(model, criterion, optimizer, patience, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 


    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    torch.backends.cudnn.benchmark = True

    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(num_epochs):
        print('-' * 10)

        ###################
        # train the model #
        ###################
        model.train() # Set model to training mode
        
        running_corrects, train_acc= 0, 0

        # Iterate over data.
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            
            #普通はこちらを使う
            optimizer.zero_grad()

            
            # Runs the forward pass with autocasting.
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            scaler.step(optimizer)

            # Updates the scale for next iteration.
            scaler.update()

            # record training loss
            train_losses.append(loss.item())

            running_corrects += torch.sum(preds==labels)

        #print()   
        train_acc = running_corrects.item()/len(train_dataset)

        #####################
        # validate the model#
        #####################

        model.eval()   # Set model to evaluate mode

        running_corrects, val_acc= 0, 0

        # Iterate over data.
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
           
            """
            print("preds:"+str(preds))
            print("labels:"+str(labels))
            print("running_corrects: "+str(str(running_corrects)))
            """

            valid_losses.append(loss.item())

            running_corrects += torch.sum(preds==labels)
        val_acc = running_corrects.item()/len(val_dataset)



        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(num_epochs))
        
        print_msg = (f'Epoch: [{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}' +'\n'
                     f'train_loss: {train_loss:.5f} ' +
                     f'train_acc: {train_acc:.5f}' +'\n'
                     f'valid_loss: {valid_loss:.5f} ' +
                     f'valid_acc: {val_acc:.5f}')
        print(print_msg)

        """
        #Scheduler step for ReduceLROnPlateau
        scheduler.step(valid_loss)
        """

        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        

        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        print('')

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))

    return model, avg_train_losses, avg_valid_losses



#Visualize model
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


In [105]:
from sklearn.model_selection import GroupKFold

PX = 224 #画像のサイズ
TRAIN_NORMALIZE_PARAM = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
TRAIN_CROP_SCALE =(0.9,1.0)
#TRAIN_BRIGHTNESS_PARAM = 0.2
#TRAIN_CONTRAST_PARAM = 0.1
#TRAIN_SATURATION_PARAM = 0.1
TRAIN_RANDOM_ROTATION = 15
TRAIN_HUE_PARAM = 0.02
VAL_NORMALIZE_PARAM = [0.494, 0.296, 0.197], [0.14,  0.114, 0.072]


# define image augmentation
train_data_transforms = transforms.Compose([
                transforms.RandomRotation(degrees=TRAIN_RANDOM_ROTATION),
                transforms.RandomResizedCrop(PX, scale=TRAIN_CROP_SCALE),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_data_transforms = transforms.Compose([
                transforms.RandomRotation(degrees=TRAIN_RANDOM_ROTATION),
                transforms.Resize(PX),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_data_transforms = transforms.Compose([
                transforms.Resize(PX),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

image_dir = "/content/drive/MyDrive/Deep_learning/Congenital_Glaucoma/glaucoma_cornea_ellipse/images"
train_X = [f"{image_dir}/{basename}" for basename in df["name"]]
groups = df["patient"]

gkf = GroupKFold(n_splits = 5)

for train_idx, val_idx in gkf.split(train_X, groups=groups):
    train_list = [train_X[i] for i in train_idx]
    val_list = [train_X[i] for i in val_idx]


#define dataset and dataloader
train_dataset = SimpleImageDataset(train_list, df, train_data_transforms)
val_dataset = SimpleImageDataset(val_list, df, val_data_transforms)
#test_dataset = SimpleImageDataset(test_list, df, test_data_transforms)

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True, pin_memory=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = True, pin_memory=True, num_workers=0)
#test_loader = DataLoader(val_dataset, batch_size = 1, shuffle = False, pin_memory=True, num_workers=0)



In [111]:
!pip install timm --q
import timm

model_ft = timm.create_model("mobilenetv3_large_100", pretrained = True, num_classes = 5)

#GPU使用
model_ft = model_ft.to(device)

#損失関数を定義
criterion = nn.BCELoss()

!pip install ranger_adabelief --q
from ranger_adabelief import RangerAdaBelief
optimizer_ft = RangerAdaBelief(model_ft.parameters(), lr=1e-3, eps=1e-12, betas=(0.9,0.999))

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


In [121]:
model_ft, train_loss, valid_loss = train_model(model_ft, criterion, optimizer_ft,  patience=30, num_epochs=300)


----------


ValueError: ignored